## Import the labrary and define the function

In [1]:
import glob
import csv
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
## define a function to translate the string data into a float list

def str2array(string):    
    together = ''
    for i in string:
        if(i != '['):
            together += i
    
    first = together.split("],")
    ret = []
    
    for i in range(len(first)):
        if(i==len(first)-1):
            second = ''
            for j in first[i]:
                if(j!=']'):
                    second += j
            second = second.split(",")
        else:
            second = first[i].split(",")
        ret.append([float(second[0]), float(second[1])])
    return ret

## define a function for exploring the maximum and minimum length of the track data

def find_maxmin(data):
    maximum = 0
    minimum = 99999
    for track in data:
        if(len(track) > maximum):
            maximum = len(track)
        if(len(track) < minimum):
            minimum = len(track)
    return maximum, minimum

## def a functino for exploring the distribution of the track length

def count(data, number = 10, maximum = 4000):
    count_num = []
    part = maximum//number
    level = part*np.array(range(1+number))
    print("level is:", level)
    
    for i in range(number):
        count_num.append(0)
    
    for line in data:
        for i in range(len(level)-1):
            if((len(line) < level[i+1]) & (len(line) >= level[i])):
                count_num[i] += 1
                continue
    return count_num

## define a function for exploring the number of the track which is above a value

def above(data, level):
    count = 0
    for line in data:
        if(len(line)>=level):
            count += 1            
    return count

## define a function for selecting track data

def select_data(track, place):
    ret = []
    for i in place:
        ret.append(track[i])
    return ret

def pick_data(data, data_length):
    ret_train = []
    ret_label = []
    n = 0
    for i in range(len(data)):
        if(len(data[i]) > data_length):
            split = len(data[i])//data_length
            ret_train.append([])
            for j in range(data_length):
                ret_train[n].append(data[i][split*j][0])
                ret_train[n].append(data[i][split*j][1])
            ret_label.append(data[i][len(data[i])-1])
            n += 1
    return ret_train, ret_label

## Import, process and save the data

In [12]:
## read in the data and seperate it into two parts
## (one set is the fully recorded data, the other one is the set of not complete data)
files = glob.glob("./split_data/part*.csv")
data_false = []
data_true = []
for table in files:
    with open(table,'r') as file:
        lines = list(csv.reader(file))
        for i in range(1, len(lines)):
            if(len(lines[i])==0):
                continue
            if(lines[i][7]=="False"):
                if(len(lines[i][8])==2):
                    data_false.append([])
                else:
                    track = str2array(lines[i][8])
                    data_false.append(track)
            elif(lines[i][7]=="True"):
                if(len(lines[i][8])==2):
                    data_true.append([])
                else:
                    track = str2array(lines[i][8])
                    data_true.append(track)

In [13]:
## save the data into a pickle file
data_dict = {'data_true': data_true, 'data_false': data_false}
with open('./trackdata.pickle','wb') as file:
    pickle.dump(data_dict, file)

In [5]:
with open('./trackdata.pickle', 'rb') as file:
    data_dict = pickle.load(file)
    data_true = data_dict['data_true']
    data_false = data_dict['data_false']    

## Explore the maximum and minimum length of the track data

In [5]:
max_false, min_false = find_maxmin(data_false)
max_true, min_true = find_maxmin(data_true)

print("Track in data_false:")
print("Maximum: %d, minimum: %d" % (max_false, min_false))

print("Track in data_true:")
print("Maximum: %d, minimum: %d" % (max_true, min_true))

Track in data_false:
Maximum: 3881, minimum: 0
Track in data_true:
Maximum: 483, minimum: 1


## Explore the distribution of the track length

In [6]:
num_false = count(data_false)
print(num_false)

level is: [   0  400  800 1200 1600 2000 2400 2800 3200 3600 4000]
[1707207, 2808, 437, 105, 58, 26, 12, 2, 2, 3]


In [7]:
print("The amount of the data_false is:", len(data_false))

The amount of the data_false is: 1710660


In [8]:
num_true = count(data_true, maximum = 500)
print(num_true)
print("The amount of the data_true is:", len(data_true))

level is: [  0  50 100 150 200 250 300 350 400 450 500]
[6, 2, 0, 0, 0, 1, 0, 0, 0, 1]
The amount of the data_true is: 10


## FInd out how many tracks are about a value

In [28]:
print("Analyse the track data without missing points......")
print("The number of the track data without missing points is:", len(data_false))
print()
level_false = [10, 15, 20, 30, 50, 60, 70, 80, 90, 100, 150, 200, 300]
for a in level_false:
    number = above(data_false, a)
    percent = number / len(data_false)
    print("The number of track with the length above %d in data_true is: %d, %.2f%%" % 
          ( a, number, percent*100))

Analyse the track data without missing points......
The number of the track data without missing points is: 1710660

The number of track with the length above 10 in data_true is: 1641024, 95.93%
The number of track with the length above 15 in data_true is: 1600460, 93.56%
The number of track with the length above 20 in data_true is: 1514478, 88.53%
The number of track with the length above 30 in data_true is: 1233766, 72.12%
The number of track with the length above 50 in data_true is: 628873, 36.76%
The number of track with the length above 60 in data_true is: 416203, 24.33%
The number of track with the length above 70 in data_true is: 273018, 15.96%
The number of track with the length above 80 in data_true is: 183744, 10.74%
The number of track with the length above 90 in data_true is: 128391, 7.51%
The number of track with the length above 100 in data_true is: 93798, 5.48%
The number of track with the length above 150 in data_true is: 30296, 1.77%
The number of track with the length

In [29]:
print("Analyse the track data with missing points......")
print("The number of the track data with missing points is:", len(data_true))
print()

level_true = [10, 20, 30, 50, 60, 300]
for a in level_true:
    number = above(data_true, a)
    percent = number / len(data_true)
    print("The number of track with the length above %d in data_true is: %d, %.2f%%" % 
          ( a, number, percent*100))

Analyse the track data with missing points......
The number of the track data with missing points is: 10

The number of track with the length above 10 in data_true is: 8, 80.00%
The number of track with the length above 20 in data_true is: 6, 60.00%
The number of track with the length above 30 in data_true is: 5, 50.00%
The number of track with the length above 50 in data_true is: 4, 40.00%
The number of track with the length above 60 in data_true is: 2, 20.00%
The number of track with the length above 300 in data_true is: 1, 10.00%


## Selecte the data from the track

In [6]:
## define the index array for selecting
data_length = 10

train_data, train_label = pick_data(data_false, data_length)
print("The number of useful data is:", len(train_data))

The number of useful data is: 1635931


In [7]:
print(len(train_data), len(train_label))

1635931 1635931


In [8]:
## save the data into a pickle file
data_dict = {'train_data10': train_data, 'train_label10': train_label}
with open('./train_data.pickle','wb') as file:
    pickle.dump(data_dict, file)

In [9]:
with open('./train_data.pickle', 'rb') as file:
    data_dict = pickle.load(file)
    train_data = data_dict['train_data10']
    train_label = data_dict['train_label10']    

In [38]:
print(train_data[0])

[[-8.618643, 41.141412], [-8.620326, 41.14251], [-8.623953, 41.144373], [-8.627373, 41.144697], [-8.632746, 41.14692], [-8.629938, 41.150385], [-8.629128, 41.15124], [-8.628687, 41.152374], [-8.630838, 41.15268], [-8.631144, 41.154489]]


In [10]:
## divide the area into n parts, only predict the area for the destination
max_x = -99999.;
max_y = -99999.;
min_x = 99999.;
min_y = 99999.;

for point in train_label:
    if max_x < point[0]: max_x = point[0]
    if max_y < point[1]: max_y = point[1]
    if min_x > point[0]: min_x = point[0]
    if min_y > point[1]: min_y = point[1]

In [11]:
print(min_x, min_y, max_x, max_y)

-13.476465 38.53413 -5.793111 44.119224


In [12]:
div_len = 0.15
area = []
labels = []
x = min_x
y = min_y
i = 0
while(x < max_x + div_len):
    area.append([])
    y = min_y
    while(y < max_y + div_len):
        area[i].append([x, y])
        y += div_len
    i += 1
    x += div_len

In [13]:
print("%d rows, %d columns" % (len(area),len(area[0])))
print("Together %d areas" % (len(area) * len(area[0])))

53 rows, 39 columns
Together 2067 areas


In [14]:
print(area[0])
print()
print(area[39])

[[-13.476465, 38.53413], [-13.476465, 38.684129999999996], [-13.476465, 38.834129999999995], [-13.476465, 38.98412999999999], [-13.476465, 39.13412999999999], [-13.476465, 39.28412999999999], [-13.476465, 39.43412999999999], [-13.476465, 39.58412999999999], [-13.476465, 39.734129999999986], [-13.476465, 39.884129999999985], [-13.476465, 40.03412999999998], [-13.476465, 40.18412999999998], [-13.476465, 40.33412999999998], [-13.476465, 40.48412999999998], [-13.476465, 40.63412999999998], [-13.476465, 40.784129999999976], [-13.476465, 40.934129999999975], [-13.476465, 41.08412999999997], [-13.476465, 41.23412999999997], [-13.476465, 41.38412999999997], [-13.476465, 41.53412999999997], [-13.476465, 41.68412999999997], [-13.476465, 41.834129999999966], [-13.476465, 41.984129999999965], [-13.476465, 42.13412999999996], [-13.476465, 42.28412999999996], [-13.476465, 42.43412999999996], [-13.476465, 42.58412999999996], [-13.476465, 42.73412999999996], [-13.476465, 42.884129999999956], [-13.4764

In [15]:
for point in train_label:
    find = False    
    for i in range(len(area) - 1):
        if((point[0] >= area[i][0][0]) & (point[0] < area[i+1][0][0])):
            for j in range(len(area[i]) - 1):
                if((point[1] >= area[i][j][1]) & (point[1] < area[i][j+1][1])):
                    find = True 
                    labels.append((i,j))
                    break
            if find: break
    if not find:
        print("Error at i = %d, j = %d" % (i, j))
        print("Point:", point)
        break

In [16]:
print(len(labels), len(train_data), len(train_label))

1635931 1635931 1635931


In [17]:
for i in range(10):
    print(train_label[i])
    print(train_data[i])
    print()

[-8.630838, 41.154489]
[-8.618643, 41.141412, -8.620326, 41.14251, -8.623953, 41.144373, -8.627373, 41.144697, -8.632746, 41.14692, -8.629938, 41.150385, -8.629128, 41.15124, -8.628687, 41.152374, -8.630838, 41.15268, -8.631144, 41.154489]

[-8.66574, 41.170671]
[-8.639847, 41.159826, -8.640351, 41.159871, -8.642196, 41.160114, -8.644455, 41.160492, -8.646921, 41.160951, -8.649999, 41.161491, -8.653167, 41.162031, -8.656434, 41.16258, -8.660178, 41.163192, -8.663112, 41.163687]

[-8.61597, 41.14053]
[-8.612964, 41.140359, -8.618472, 41.141412, -8.6319, 41.146461, -8.62875, 41.152662, -8.642205, 41.154021, -8.649837, 41.1543, -8.65008, 41.154291, -8.649198, 41.152374, -8.647587, 41.148405, -8.635338, 41.147964]

[-8.607996, 41.142915]
[-8.574678, 41.151951, -8.574723, 41.151933, -8.577135, 41.150232, -8.580744, 41.14503, -8.610012, 41.146479, -8.587197, 41.149224, -8.592543, 41.146614, -8.598816, 41.146101, -8.602785, 41.145705, -8.605854, 41.142555]

[-8.687268, 41.178087]
[-8.645994, 

In [18]:
print(labels[2000:2200])
print(point)

[(32, 17), (32, 17), (32, 17), (32, 17), (32, 18), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 18), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (31, 17), (37, 13), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 16), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 16), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 18), (32, 17), (32, 17), (32, 17), (31, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 17), (32, 15), (32, 17), (32, 17), (32, 17), (32, 17), (32, 16), (32, 17), (32, 18), (32, 17),

In [19]:
## cause I find the area (32,17) is very commen, I need to analyse the reapeat area and the count number of them.
count = []
index_set = []
for index in range(len(labels)):
    add = True
    for i in range(len(index_set)):
        if((index_set[i][0] == labels[index][0]) & (index_set[i][1] == labels[index][1])):
            add = False
            count[i] += 1
            break
    if add:
        index_set.append(labels[index])
        count.append(0)

In [20]:
print("number of the area:", len(index_set))
print()
print(index_set)
print(count)

number of the area: 176

[(32, 17), (31, 17), (33, 17), (32, 18), (32, 16), (31, 18), (33, 18), (32, 19), (37, 13), (32, 15), (33, 16), (35, 17), (34, 17), (38, 18), (28, 1), (35, 18), (34, 19), (33, 19), (32, 14), (31, 19), (30, 10), (30, 3), (34, 18), (33, 21), (33, 15), (33, 13), (33, 11), (31, 21), (43, 20), (43, 19), (35, 19), (33, 20), (32, 21), (39, 20), (30, 8), (38, 19), (31, 20), (33, 12), (41, 15), (36, 18), (32, 22), (30, 21), (31, 14), (34, 20), (32, 12), (37, 21), (32, 13), (31, 22), (32, 20), (36, 15), (34, 16), (36, 14), (36, 19), (29, 1), (32, 23), (38, 17), (33, 22), (38, 13), (37, 17), (32, 11), (33, 23), (33, 14), (32, 7), (39, 8), (42, 18), (31, 13), (35, 21), (27, 1), (39, 18), (36, 17), (34, 12), (35, 16), (44, 13), (35, 14), (35, 20), (32, 24), (40, 21), (31, 7), (37, 0), (32, 9), (41, 21), (31, 12), (36, 20), (41, 19), (41, 13), (32, 10), (37, 14), (36, 13), (34, 13), (34, 21), (31, 11), (40, 19), (31, 8), (32, 8), (48, 20), (47, 20), (32, 5), (35, 12), (31, 9)

## Split data into train, validation and test data

In [21]:
labels_for_train = []
for i in range(53):
    for j in range(39):
        labels_for_train.append((i, j))

In [22]:
print(labels_for_train[:100])

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21)]


In [23]:
Y = []

for point in labels:
    find = False
    for i in range(len(labels_for_train)):
        if((point[0] == labels_for_train[i][0]) & (point[1] == labels_for_train[i][1])):
            Y.append(i)
            find = True
            break
    if not find:
        print("error")
        break

In [24]:
print(Y[:10])

[1265, 1265, 1265, 1265, 1226, 1265, 1265, 1265, 1265, 1265]


In [25]:
x_train, x_test, y_train, y_test = train_test_split(train_data, Y, test_size = 0.3, random_state = 42)

In [26]:
## save the data into a pickle file
# need to save: data, area, labels_for_train
data_dict = {'x_train': x_train, 'x_test': x_test, 'y_train': y_train, 'y_test': y_test, "area": area,
            "labels_for_train": labels_for_train}
with open('./train_data/train_for_classifier_uniform_area.pickle','wb') as file:
    pickle.dump(data_dict, file)

In [ ]:
with open('././train_data/train_for_classifier_uniform_area.pickle', 'rb') as file:
    data_dict = pickle.load(file)
    x_train = data_dict['x_train']
    x_test = data_dict['x_test']    
    y_train = data_dict['y_train']
    y_test = data_dict['y_test']
    area = data_dict['area']
    labels_for_train = data_dict['labels_for_train']